# Orthotropic linear elasticity

The original example is taken from [here](https://comet-fenics.readthedocs.io/en/latest/demo/elasticity/orthotropic_elasticity.py.html). This example just updates code to FEniCSx

## Introduction

In this numerical tour, we will show how to tackle the case of orthotropic elasticity (in a 2D setting).

We consider here the case of a square plate perforated by a circular hole of radius R
, the plate dimension is $2L×2L$
 with $L≫R$
Only the top-right quarter of the plate will be considered. Loading will consist of a uniform traction on the top/bottom boundaries, symmetry conditions will also be applied on the correponding symmetry planes. The geometry is shown on figure

<div style="text-align:center">
  <img src="pics/geometry.png" alt="Problem geometry" width=400 height=400 title="Problem geometry">
  <figcaption><b>Problem geometry</b></figcaption>
</div>

In [21]:
import numpy as np
import ufl

from mpi4py import MPI
from petsc4py.PETSc import ScalarType

from dolfinx import mesh, fem, plot, io
from dolfinx.io import XDMFFile, gmshio
from dolfinx.mesh import DiagonalType
import gmsh

import pyvista

Generation of domain is performed using gmsh. 
At first we generate points, than lines and circle arc between them, then they are conecting into curve loop, which is use to generate cloe plane.

In [23]:
L, R = 1., 0.1
N = 50 # mesh density


SHOW_PYVISTA = False;
# SHOW_PYVISTA = True;

gmsh.initialize();
model = gmsh.model();
model.add("main_domain");
model.setCurrent("main_domain");

try:
    p1 = model.occ.add_point(0, 0, 0);
    p2 = model.occ.add_point(L, 0, 0);
    p3 = model.occ.add_point(L, L, 0);
    p4 = model.occ.add_point(0, L, 0);

    p5 = model.occ.add_point(R, 0, 0);
    p6 = model.occ.add_point(0, R, 0);

    l1 = model.occ.add_line(p5, p2);
    l2 = model.occ.add_line(p2, p3);
    l3 = model.occ.add_line(p3, p4);
    l4 = model.occ.add_line(p4, p6);
    # l5 = model.occ.add_line(p6, p5);
    ar = model.occ.add_circle_arc(p5, p1, p6); 

    curve_loop = model.occ.add_curve_loop([l1, l2, l3, l4, ar]);
    model.occ.synchronize();

    domain = model.occ.add_plane_surface([curve_loop]);

    model.occ.synchronize();

    model.add_physical_group(dim=2, tags=[domain]);
    gmsh.option.setNumber("Mesh.Algorithm", 8);

    # Generate the mesh
    model.mesh.set_size(model.getEntities(0), 0.05);
    model.mesh.set_size([(0, p5),(0, p6)], 0.01);
    model.mesh.generate(dim=2);
    model.mesh.recombine();

    # Create a DOLFINx mesh (same mesh on each rank)
    msh, cell_markers, facet_markers = gmshio.model_to_mesh(model, MPI.COMM_SELF,0,gdim=2);
    msh.name = "Box";
    cell_markers.name = f"{msh.name}_cells";
    facet_markers.name = f"{msh.name}_facets";
finally:
    gmsh.finalize();

if SHOW_PYVISTA:
    
    pyvista.start_xvfb();
    plotter = pyvista.Plotter();

    topology, cell_types, geometry = plot.create_vtk_mesh(msh);
    grid = pyvista.UnstructuredGrid(topology, cell_types, geometry);
    # grid.point_data["u"] = np.c_[uh.x.array.reshape((geometry.shape[0], 2)), np.zeros(geometry.shape[0]).T]
    actor_0 = plotter.add_mesh(grid, style="wireframe", color="k");
    # warped = grid.warp_by_vector("u", factor=500)
    # actor_1 = plotter.add_mesh(warped, show_edges=True)

    plotter.show_axes()
    if not pyvista.OFF_SCREEN:
        plotter.show();
    else:
        figure = plotter.screenshot("fundamentals_mesh.png");

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 20%] Meshing curve 2 (Line)
Info    : [ 40%] Meshing curve 3 (Line)
Info    : [ 60%] Meshing curve 4 (Line)
Info    : [ 80%] Meshing curve 5 (Circle)
Info    : Done meshing 1D (Wall 0.00478031s, CPU 0.006092s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay for Quads)
Info    : Done meshing 2D (Wall 0.0483795s, CPU 0.050328s)
Info    : 1034 nodes 2070 elements
Info    : Recombining 2D mesh...
Info    : Blossom: 2836 internal 129 closed
Info    : Blossom recombination completed (Wall 0.0206078s, CPU 0.020616s): 961 quads, 0 triangles, 0 invalid quads, 0 quads with Q < 0.1, avg Q = 0.874401, min Q = 0.42561
Info    : Done recombining 2D mesh (Wall 0.0207735s, CPU 0.02107s)


## Constitutive relation
Constitutive relations will be defined using an engineering (or Voigt) notation (i.e. second order tensors will be written as a vector of their components) contrary to the 2D linear elasticity example which used an intrinsic notation. In the material frame, which is assumed to coincide here with the global ($Oxy$) frame, the orthotropic constitutive law writes $\mathbf{\varepsilon}=\mathbf{S\sigma}$ using the compliance matrix $\mathbf{S}$ with:

with $E_x$, $E_y$ the two Young’s moduli in the orthotropy directions, $\nu_{xy}$ the in-plane Poisson ration (with the following relation ensuring the constitutive relation symmetry $(\nu_{yx}=\nu_{xy}E_y/E_x)$ and $G_{xy}$ being the shear modulus. This relation needs to be inverted to obtain the stress components as a function of the strain components $\mathbf{\sigma}=\mathbf{C\varepsilon}$ with $\mathbf{C}=\mathbf{S}^{−1}$:

In [24]:
Ex, Ey, nuxy, Gxy = 100., 10., 0.3, 5.;
S = ufl.as_matrix([[   1./Ex, -nuxy/Ex,     0.],
                   [-nuxy/Ex,    1./Ey,     0.],
                   [      0.,       0., 1./Gxy]
                  ])
C = ufl.inv(S)

> 
> **NOTE**
>
> Here we used the `ufl.inv` opertor to compute the elasticity matrix C. We could also have computed analytically the inverse relation. Note that the `ufl.inv` operator is implemented only up to 3x3 matrices. Extension to the 3D case yields 6x6 matrices and therefore requires either analytical inversion or numerical inversion using *Numpy* for instance (assuming that the material parameters are constants).
>

We define different functions for representing the stress and strain either as second-order tensor or using the Voigt engineering notation:

In [25]:
def eps(v):
    return ufl.sym(ufl.grad(v));

def strain2voigt(e):
    """e is a 2nd-order tensor, returns its Voigt vectorial representation"""
    return ufl.as_vector([e[0,0],e[1,1],2*e[0,1]]);

def voigt2stress(s):
    """
    s is a stress-like vector (no 2 factor on last component)
    returns its tensorial representation
    """
    return ufl.as_tensor([[s[0], s[2]],
                          [s[2], s[1]]
                         ]);
# notice, that C - is global variable, defined in previous code cell
def sigma(v):
    return voigt2stress(ufl.dot(C, strain2voigt(eps(v))));

## Problem position and resolution

Different parts of the quarter plate boundaries are now defined as well as the exterior integration measure `ds`:

In [ ]:
# Define function space
V = fem.VectorFunctionSpace(msh, ("CG", 2));

# Define variational problem
du = ufl.TrialFunction(V);
u_ = ufl.TestFunction(V);

a = ufl.inner(sigma(du), eps(u_))*ufl.dx
# uniform traction on top boundary
T = fem.Constant(msh, (0, 1e-3))

ds = ufl.Measure("ds", domain=msh)

l = ufl.inner(T, u_)*ufl.dx

Apply boundary conditions. Symmetric boundary conditions are applied on the **bottom** and **left** boundaries and the problem is solved:

In [ ]:
def top(x):
    return np.isclose(x[1], L);

def left(x):
    return np.isclose(x[0], 0.);

def bottom(x):
    return np.islose(x[1], 0);


fdim = fe_mesh.topology.dim - 1
left_facets = mesh.locate_entities_boundary(fe_mesh, fdim, left)

u_D = np.array([0.,0.], dtype=ScalarType)
bc = fem.dirichletbc(u_D, fem.locate_dofs_topological(V, fdim, left_facets), V)